In [69]:
import pandas as pd
import json
import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np
import random
import string
import pickle


from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

In [70]:
intents = json.loads(open('data-flair/intents.json').read())

In [78]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'context': ['']},
  {'tag': 'options',
   'patterns': ['How you could help me?',
    'What you can do

In [73]:
vocab = []
documents = []
classes = []
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word = nltk.word_tokenize(pattern)
        vocab.extend(word)
        documents.append((word,intent['tag']))
        #print(documents)
        
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [74]:
#vocab

In [75]:
lemmatizer = WordNetLemmatizer()

vocab = [lemmatizer.lemmatize(word.lower()) for word in vocab if word not in nltk.word_tokenize(string.punctuation)]
vocab = sorted(list(set(vocab)))
classes = sorted(list(set(classes)))
print(len(documents))
print(len(classes))
print(len(vocab))

pickle.dump(vocab,open('dataflair_vocab.pkl','wb'))
pickle.dump(classes,open('dataflair_classes.pkl','wb'))

47
9
87


In [76]:
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag=[]
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for word in vocab:
        bag.append(1) if word in pattern_words else bag.append(0)
        output_row = list(output_empty)
        #print(doc[1])
        output_row[classes.index(doc[1])] = 1
    training.append([bag,output_row])

random.shuffle(training)
training = np.array(training, dtype=object)
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [57]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),),activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation = 'softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum = 0.9, nesterov= True)
model.compile(loss='categorical_crossentropy', optimizer = 'sgd', metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size= 5, verbose=1)
model.save('dataflair-chatbot.h5', hist)
print('done')

Epoch 1/200
10/10 [==============================] - 0s 786us/step - loss: 2.2360 - accuracy: 0.0851
Epoch 2/200
10/10 [==============================] - 0s 882us/step - loss: 2.2519 - accuracy: 0.0638
Epoch 3/200
10/10 [==============================] - 0s 716us/step - loss: 2.2543 - accuracy: 0.1064
Epoch 4/200
10/10 [==============================] - 0s 723us/step - loss: 2.2544 - accuracy: 0.1064
Epoch 5/200
10/10 [==============================] - 0s 937us/step - loss: 2.2175 - accuracy: 0.1064
Epoch 6/200
10/10 [==============================] - 0s 685us/step - loss: 2.1655 - accuracy: 0.1064
Epoch 7/200
10/10 [==============================] - 0s 716us/step - loss: 2.1911 - accuracy: 0.2553
Epoch 8/200
10/10 [==============================] - 0s 837us/step - loss: 2.1543 - accuracy: 0.0851
Epoch 9/200
10/10 [==============================] - 0s 717us/step - loss: 2.1548 - accuracy: 0.2128
Epoch 10/200
10/10 [==============================] - 0s 718us/step - loss: 2.1375 - accura

In [58]:
def cleanup_sentence(sentence):
    sw = nltk.word_tokenize(sentence)
    sw = [lemmatizer.lemmatize(word.lower()) for word in sw]
    return sw

def bagofwords(sentence, words, show_details = True):
    sentence_words = cleanup_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w==s:
                bag[i] = 1
                if show_details:
                    print('found in bag: %s' % w)
    return(np.array(bag))

In [59]:
p=bagofwords("how are you",vocab)

found in bag: how
found in bag: are
found in bag: you


In [60]:
from keras.models import load_model
modelx = load_model('dataflair-chatbot.h5')

def predict_class(sentence, model):
    bow = bagofwords(sentence,vocab,show_details=False)
    res = model.predict(np.array([bow]))[0]
    error_threshold = 0.25
    results = [[i,r] for i,r in enumerate(res) if r > error_threshold]
    results.sort(key=lambda x: x[1], reverse= True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]],'probability': str(r[1])})
    return return_list

In [61]:
predict_class("how are you", modelx)

[{'intent': 'greeting', 'probability': '0.6777462'}]

In [62]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if (i['tag'] == tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_respone(text):
    ints = predict_class(text,model)
    res = getResponse(ints, intents)
    return res

In [68]:
chatbot_respone('what can you do')

'Offering support for Adverse drug reaction, Blood pressure, Hospitals and Pharmacies'